In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import time
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, RidgeCV
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,root_mean_squared_error
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor

from utils import clean_data
from utils import split_trajectories
from utils import replicate_initial_position_by_block
from utils import get_n_trajectories
from utils import plot_y_yhat

In [ ]:
train = pd.read_csv('../data/X_train.csv')
test =  pd.read_csv('../data/X_test.csv')
sample_submission = pd.read_csv('../data/sample_submission.csv')#This is a file with random numbers as predictions
                                                                #dans le futur fichier résultat ne pas mettre l'index

In [ ]:
train_cleaned=clean_data(train)

In [ ]:
train_after_split, validation_after_split, test_after_split = split_trajectories(train_cleaned)

In [ ]:
train_entry=replicate_initial_position_by_block(train_after_split)
validation_entry=replicate_initial_position_by_block(validation_after_split)
test_entry=replicate_initial_position_by_block(test_after_split)

In [ ]:
input_cols = ['t', 'x_1', 'y_1', 'x_2', 'y_2', 'x_3', 'y_3']
target_cols = ['x_1', 'y_1', 'x_2', 'y_2', 'x_3', 'y_3']
X_train = train_entry[input_cols]          
y_train = train_after_split[target_cols] 

X_val = validation_entry[input_cols]
y_val = validation_after_split[target_cols]

X_test = test_entry[input_cols]
y_test= test_after_split[target_cols]

In [ ]:
nb_traj=10
#train_entry_subset= get_n_trajectories(train_entry, nb_traj)
#train_entry_subset = train_entry_subset[['t', 'x_1', 'y_1', 'x_2', 'y_2', 'x_3', 'y_3']]

#train_after_split_subset= get_n_trajectories(train_after_split, nb_traj)
#train_after_split_subset = train_after_split_subset[['x_1', 'y_1', 'x_2', 'y_2', 'x_3', 'y_3']]

#validation_entry_subset = get_n_trajectories(validation_entry, nb_traj)
#validation_entry_subset = validation_entry_subset[['t', 'x_1', 'y_1', 'x_2', 'y_2', 'x_3', 'y_3']]

#validation_after_split_subset = get_n_trajectories(validation_after_split, nb_traj)
#validation_after_split_subset = validation_after_split_subset[['x_1', 'y_1', 'x_2', 'y_2', 'x_3', 'y_3']]

In [ ]:
def validate_poly_regression(X_train, y_train, X_val, y_val, regressor=None, degrees=range(1,5), max_features=None):
    """
    Validate polynomial regression models with optional regularization.
    Returns best model, RMSE, degree, regularization type, and alpha.
    Also prints elapsed time.
    """
    start_time = time.time()  # start timing

    best_rmse = np.inf
    best_model = None
    best_degree = None
    best_alpha = None
    best_reg = None

    alphas = np.logspace(-5, 5, 11)

    for degree in tqdm(degrees, desc="Testing degrees"):
        poly = PolynomialFeatures(degree=degree, include_bias=False)
        poly.fit(X_train)
        n_features = poly.n_output_features_
        print(f"Degree {degree}: {n_features} features")

        if max_features and n_features > max_features:
            print(f" Skipped degree {degree} (too many features: {n_features})")
            continue

        for alpha in alphas:
            if regressor == 'L1':
                model = Lasso(alpha=alpha, max_iter=5000, random_state=42)
            elif regressor == 'L2':
                model = Ridge(alpha=alpha, random_state=42)
            else:
                model = LinearRegression()
                alpha = None  # pas de régularisation
                regressor = 'None'

            pipe = Pipeline([
                ('scaler', StandardScaler()),
                ('poly', poly),
                ('reg', model)
            ])

            pipe.fit(X_train, y_train)
            y_pred = pipe.predict(X_val)
            rmse = root_mean_squared_error(y_val, y_pred)

            if rmse < best_rmse:
                best_rmse = rmse
                best_model = pipe
                best_degree = degree
                best_alpha = alpha
                best_reg = regressor

    elapsed_time = time.time() - start_time  # end timing
    print(f"\n⏱️ Total elapsed time: {elapsed_time:.2f} seconds")
    print(f"✅ Best model: degree={best_degree}, RMSE={best_rmse:.4f}, reg={best_reg}, alpha={best_alpha}")

    return best_model, best_rmse, best_degree, best_reg, best_alpha

In [ ]:
best_model_L1, best_rmse_L1, best_degree_L1, best_reg_L1, best_alpha_L1 = validate_poly_regression(
   X_train, y_train,X_val,y_val, regressor='L1'
)

best_model_L2, best_rmse_L2, best_degree_L2, best_reg_L2, best_alpha_L2 = validate_poly_regression(
   X_train, y_train,X_val,y_val, regressor='L2'
)

best_model_L, best_rmse_L, best_degree_L, best_reg_L, best_alpha_L = validate_poly_regression(
   X_train, y_train,X_val,y_val
)

In [ ]:
y_pred = best_model_L2.predict(X_val)
plot_y_yhat(y_val.values, y_pred, plot_title="Best Model Predictions")

In [ ]:
X_test = test[['t','x0_1','y0_1','x0_2','y0_2','x0_3','y0_3']].copy()
X_test.columns = ['t','x_1','y_1','x_2','y_2','x_3','y_3'] 


ids = test['Id']


predictions = best_model_L2.predict(X_test)

target_cols = ['x_1','y_1','x_2','y_2','x_3','y_3']
output_df = pd.DataFrame(predictions, columns=target_cols)
output_df.insert(0, 'Id', ids)


output_df.to_csv('augmented_polynomial_submission.csv', index=False)